In [0]:
! yes | pip3 uninstall opencv-python
! yes | pip3 uninstall opencv-contrib-python
! yes | pip3 install opencv-python==3.4.2.16
! yes | pip3 install opencv-contrib-python==3.4.2.16 

In [0]:
#Connect to Goole Mount
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#for using GPU, Install kmc2
!pip install kmc2

In [0]:
from sklearn.model_selection import train_test_split
from imutils import paths
import numpy as np
import imutils 
import cv2 
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import copy
import pickle
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn import preprocessing
#from libKMCUDA import kmeans_cuda # kmeans-gpu 사용 --> not installed

파일처리 해주기 
Label과 Data  매칭시키기

In [0]:
df_data=pd.read_csv('/content/drive/My Drive/2019-ml-finalproject/Label2Names.csv',header=None)

DATA_ROOT_TRAIN="/content/drive/My Drive/2019-ml-finalproject/train"
train_des=list()
train_labels=list()

for cls in tqdm(os.listdir(DATA_ROOT_TRAIN)): #진행도를 알려줌
  img_list = os.listdir(DATA_ROOT_TRAIN+'/'+cls)
  
  if cls == 'BACKGROUND_Google' :
    label= 101
  else :
    label=(df_data.index[df_data[1]==cls]+1).tolist()[0]
    '''
  이미지 잘들어있는지 확인
  if len(img_list) != 30:
   print(cls)
   print(len(img_list)) -> 2750장 -> 3060장으로 바꿈
   '''
  for img in img_list:
    image=cv2.imread(DATA_ROOT_TRAIN+'/'+cls+'/'+img)
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    dst = gray.copy() 
    roi = gray[16:np.shape(gray)[0]-16, 16:np.shape(gray)[1]-16]
    #gray=cv2.resize(gray, dsize=(256, 256))
    train_des.append(roi)
    train_labels.append(label)




In [0]:
#피클사용 저장
#with open('/content/drive/My Drive/2019-ml-finalproject/train_labels.pickle','wb') as f:
#  pickle.dump(train_labels,f,pickle.HIGHEST_PROTOCOL)
#with open('/content/drive/My Drive/2019-ml-finalproject/train_des.pickle','wb') as f:
  pickle.dump(train_des,f,pickle.HIGHEST_PROTOCOL)
#print(np.shape(train_labels))


In [0]:
#피클사용 로드
with open('/content/drive/My Drive/2019-ml-finalproject/train_labels.pickle', 'rb') as f:
    train_labels1 = pickle.load(f)
with open('/content/drive/My Drive/2019-ml-finalproject/train_des.pickle', 'rb') as f:
    train_des1 = pickle.load(f)

FileNotFoundError: ignored

# BOVW 사용 성능 43% (LEVEL 0)

In [0]:
#SIFT 계산
def computeSIFT(data):
    x = []
    for i in range(0, len(data)):
        sift = cv2.xfeatures2d.SIFT_create()
        img = data[i]
        step_size = 8 # stepsize : 8  range kp 설정.
        kp = [cv2.KeyPoint(x, y, step_size) for x in range(16, img.shape[0]-16, step_size) for y in range(16, img.shape[1]-16, step_size)]
        dense_feat = sift.compute(img, kp) 
        x.append(dense_feat[1]) #description 값 계산
        
        ''' weak feature --> 나쁜성능
        #keypoints, descriptors = sift.detectAndCompute(img, None)
        #x.append(descriptors)
        '''
    return x

In [0]:
#print(len(data))
final_train1=computeSIFT(train_des)  #1로 수정하기
print(np.shape(final_train1))
final_all_train_desc1 = []
for i in range(len(final_train1)):
    for j in range(final_train1[i].shape[0]):
        final_all_train_desc1.append(final_train1[i][j,:])

final_label1=train_labels1 #1로수정하기
final_all_train_desc1 = np.array(final_all_train_desc1)
print(final_all_train_desc1.shape)


In [0]:
import kmc2
from sklearn.cluster import MiniBatchKMeans
def clusterFeatures(all_train_desc, k):
    seeding = kmc2.kmc2(all_train_desc, k)
    model = MiniBatchKMeans(k, init=seeding).fit(all_train_desc)
    codebook = model.cluster_centers_
    return codebook,model
k = 200 #코드북 사이즈

In [0]:
# kmeans 생성하기
codebook,kmeans1 = clusterFeatures(final_all_train_desc1, k)
print(codebook.shape)

In [0]:
#히스토그램 생성
print(np.shape(train_des))
def formTrainingSetHistogram(x_train, kmeans, k,codebook):
    train_hist = []
    codebook=list(codebook)
    for i in range(len(x_train)):

        predict=vq(x_train[i],codebook)[0]
        train_hist.append(np.bincount(predict, minlength=k).reshape(1,-1).ravel())
        
    return np.array(train_hist) 

final_histo1=formTrainingSetHistogram(final_train1,kmeans1,k,codebook)

#테스트 데이터 *가져오기*

In [0]:
DATA_ROOT_TEST = "/content/drive/My Drive/2019-ml-finalproject/testAll_v2"
test_des=list()
img_list = os.listdir(DATA_ROOT_TEST)
print(len(img_list))
img_list.sort()


for img in tqdm(img_list):
  image=cv2.imread(DATA_ROOT_TEST+'/'+img)
  gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  dst = gray.copy() 
  roi = gray[16:np.shape(gray)[0]-16, 16:np.shape(gray)[1]-16]
  test_des.append(roi)

In [0]:
#sift한 파일 저장
#with open('/content/drive/My Drive/2019-ml-finalproject/test_des.pickle','wb') as f:
#  pickle.dump(test_des,f,pickle.HIGHEST_PROTOCOL)
#Load
with open('/content/drive/My Drive/2019-ml-finalproject/test_des.pickle', 'rb') as f:
    test_des = pickle.load(f)
    

#분류시작

In [0]:
from sklearn.svm import SVC

def histogramIntersection(M, N):
    m = M.shape[0]
    n = N.shape[0]
    result = np.zeros((m,n))
    for i in range(m):
        if i%100==1:
          print(i,"번째 실행중")
        for j in range(n):
            temp = np.sum(np.minimum(M[i], N[j]))
            result[i][j] = temp
    return result
gramMatrix = histogramIntersection(final_histo1, final_histo1)
print(gramMatrix.shape)
print(np.shape(final_label1))
final_clf = SVC(kernel='precomputed',C=1)
final_clf.fit(gramMatrix,final_label1)
predictMatrix = histogramIntersection(final_test_histo1, final_histo1)
result=final_clf.predict(predictMatrix)

In [0]:
predictMatrix = histogramIntersection(final_test_histo1, final_histo1)
result=final_clf.predict(predictMatrix)

출력 kaggle 등록 및 제출


In [0]:
result = result.reshape(-1,1)
result_img_list=np.array(img_list).reshape(-1,1)
total_result = np.hstack([result_img_list,result])

print(result)

In [0]:
#pands 이용해 변환
df=pd.DataFrame(total_result,columns=["id","Category"])
df.to_csv('results-jaeung.csv',index=False,header=True)

In [0]:

!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

# 캐글연동을 위한 토큰 입력
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


# 버전이 1.5.6 이 아니면, 진행할 수 없다
! kaggle -v


In [0]:
#kaggle 로 제출과정

! kaggle competitions submit -c 2019-ml-finalproject -f results-jaeung.csv -m "jaeung"


